In [82]:
import numpy as np
import sys
import time
import matplotlib.pyplot as plt
import itertools

# STUDENT ID NUMBER: 

# INSTRUCTIONS

1. Put your student ID number (no other identifying features).  
If collaborating, put collaborators' id numbers underneath (again, no identifying features).
1. Submit both a notebook and exported PDF of the notebook after restart-and-run-all.  
    1. Submitted notebook must run in reasonable time.
    1. Ensure the exported pdf is LaTex formatted. 
1. No formal written section, but some of the combinatorial search questions may be abbreviated by specific algorithm choices.  
Justify algorithm choices outside of brute-force search in a separate typset pdf. 
1. For the coding section:
    1. Limit code answers to the designated spaces in functions (marked off by "###") and cells (marked by "### DO: *\<instructions\>*").  
       You may add extra cells to run subroutines (in order), but only marked cells will be graded. 
    1. Do not change the provided cells, or their order.  
       Any detected alterations will zero out that section. 
       
### Tips
- You are encouraged to work either alone or with your official partner, but not more.  
- For the coding section, test and sanity-check your work extensively.  

# Coding [$4\times 25 = 100$ pts]

## Ex. 1 - N-queens

Make a function to generate all valid arrangements of $N$-queens on an $N\times N$ chess board s.t. no queens attack each other.  
Ensure your results are valid up to $N = 8$.  
Check against a known table. 

Bonus 10 points to the highest $N$ in the class that runs in under a minute (as run on my system, ties split the points).  

In [ ]:
### DO: Make a function to read a text file containing arrangements of N-queens on a board to ensure no queens attack each other.  
### Prove it works. 

def safe(row,col,l,n):

    trow=row
    tcol=col
    while row>=0 and col>=0:
        if l[row][col]=="Q":
            return False
        row-=1
        col-=1
    row=trow
    col=tcol
    while col>=0:
        if l[row][col]=="Q":
            return False
        col-=1
    col=tcol
    while row<n and col>=0:
        if l[row][col]=="Q":
            return False
        row+=1
        col-=1
    return True

def is_valid_board_configuration(board: list, n: int):
    queens_positions = []
    for r in range(n):
        for c in range(n):
            if board[r][c] == 'Q':
                queens_positions.append((r, c))

    # Check for conflicts between all pairs of queens
    num_queens = len(queens_positions)
    for i in range(num_queens):
        r1, c1 = queens_positions[i]
        for j in range(i + 1, num_queens):
            r2, c2 = queens_positions[j]

            # Check same row
            if r1 == r2:
                return False
            # Check same column
            if c1 == c2:
                return False
            # Check same diagonal
            if abs(r1 - r2) == abs(c1 - c2):
                return False
    return True

def read_and_validate_n_queens_solutions(filename: str, n: int):
    print(f"\n Validating Solutions from '{filename}' for N = {n}")
    with open(filename, 'r') as f:
        lines = f.readlines()

    current_board_lines = []
    solution_count = 0
    valid_solutions = 0

    # Helper function to process the accumulated board lines
    def process_current_board():
        nonlocal solution_count, current_board_lines, valid_solutions
        if current_board_lines: # Only process if there are lines accumulated
            solution_count += 1
            # Filter out any potential empty strings that might have snuck in (though strip() should handle most)
            board = [row for row in current_board_lines if row]
            
            # Validate board dimensions
            if len(board) != n or any(len(row) != n for row in board):
                print(f"Solution {solution_count}: Invalid board dimensions. Expected {n}x{n}, got {len(board)}x{len(board[0]) if board else 0}.")
            else:
                is_valid = is_valid_board_configuration(board, n)
                if is_valid:
                    print(f"Solution {solution_count}: VALID")
                    valid_solutions +=1
                if not is_valid:
                    print(f"Solution {solution_count}: INVALID")
                    print("-" * 10)
                    for row_str in board:
                        print(row_str)
                    print("-" * 10)
            current_board_lines = [] # Reset for next board

    for line in lines:
        line = line.strip()

        # If it's an empty line, or a known header/separator, process the accumulated board
        # This handles both the generated file format and user-provided format with empty line separators
        if not line or \
            line.startswith("N-Queens Solutions for N =") or \
            line.startswith("Total Solutions:") or \
            line.startswith("Solution ") or \
            line.startswith("="*20):
            process_current_board()
            continue # Move to the next line after processing/skipping

        # If it's a board row, add it to current_board_lines
        # Ensure it's a valid board row (only '.' or 'Q' and correct length)
        if all(char in ('.', 'Q') for char in line) and len(line) == n:
            current_board_lines.append(line)
        else:
            # Warn about unexpected lines that don't fit the board row pattern
            print(f"Warning: Skipping unexpected line in file: '{line}' (Length: {len(line)}, Expected N: {n})")
    process_current_board()
    
N = 8
txt_file_name = "NQueens_filereader_test.txt"

read_and_validate_n_queens_solutions(txt_file_name, N)




 Validating Solutions from 'NQueens_filereader_test.txt' for N = 8
Solution 1: VALID
Solution 2: VALID
Solution 3: VALID
Solution 4: INVALID
----------
Q.......
Q.......
Q.......
Q.......
Q.......
Q.......
Q.......
Q.......
----------
Solution 5: VALID
4


Proof was done by creating a txt file with all possible configurations of placing N queens   
with the condition of having one queen per row since this is a requirement for the N queens problem.

In [102]:
#Modified version of file reader to only return amount of valid and invalid solutions
def modified_read_and_validate_n_queens_solutions(filename: str, n: int):
    print(f"\n Validating Solutions from '{filename}' for N = {n}")
    with open(filename, 'r') as f:
        lines = f.readlines()

    current_board_lines = []
    solution_count = 0
    valid_solutions = 0
    invalid_solutions = 0

    # Helper function to process the accumulated board lines
    def process_current_board():
        nonlocal solution_count, current_board_lines, valid_solutions, invalid_solutions
        if current_board_lines: # Only process if there are lines accumulated
            solution_count += 1
            # Filter out any potential empty strings that might have snuck in (though strip() should handle most)
            board = [row for row in current_board_lines if row]
            
            # Validate board dimensions
            if len(board) != n or any(len(row) != n for row in board):
                print(f"Solution {solution_count}: Invalid board dimensions. Expected {n}x{n}, got {len(board)}x{len(board[0]) if board else 0}.")
            else:
                is_valid = is_valid_board_configuration(board, n)
                if is_valid:
                    valid_solutions +=1
                if not is_valid:
                    invalid_solutions +=1
            current_board_lines = [] # Reset for next board

    for line in lines:
        line = line.strip()

        # If it's an empty line, or a known header/separator, process the accumulated board
        # This handles both the generated file format and user-provided format with empty line separators
        if not line or \
            line.startswith("N-Queens Solutions for N =") or \
            line.startswith("Total Solutions:") or \
            line.startswith("Solution ") or \
            line.startswith("="*20):
            process_current_board()
            continue # Move to the next line after processing/skipping

        # If it's a board row, add it to current_board_lines
        # Ensure it's a valid board row (only '.' or 'Q' and correct length)
        if all(char in ('.', 'Q') for char in line) and len(line) == n:
            current_board_lines.append(line)
        else:
            # Warn about unexpected lines that don't fit the board row pattern
            print(f"Warning: Skipping unexpected line in file: '{line}' (Length: {len(line)}, Expected N: {n})")
    process_current_board()

    print(f"Validation Summary for '{filename}':")
    print(f"Total Valid Solutions: {valid_solutions}")
    print(f"Total Invalid Solutions: {invalid_solutions}")
    
def all_configurations_one_queen_per_row(N):

    n_value = N
    all_placements = []
    # Generate all permutations of column indices for each row
    # Each permutation represents a unique placement of 8 queens (one per row, one per column)
    for p in itertools.permutations(range(n_value)):
        board = ['.' * n_value for _ in range(n_value)]
        for row_idx, col_idx in enumerate(p):
            board[row_idx] = board[row_idx][:col_idx] + 'Q' + board[row_idx][col_idx+1:]
        all_placements.append(board)
    
    print(f"\nGenerating and Saving all {len(all_placements)} possible {n_value}-Queens placements (one per row/column)")

    filename = f"AllSolutions_8Queens.txt"
    with open(filename, 'w') as f:
            for i, board in enumerate(all_placements):
                for row_str in board:
                    f.write(row_str + "\n")
                f.write("\n")
    print(f"Successfully wrote {len(all_placements)} solutions to '{filename}'")
    print(f"Finished generating and saving all {n_value}-Queens placements.")

    return filename

one_queen_per_row_txt_file = all_configurations_one_queen_per_row(N)
modified_read_and_validate_n_queens_solutions(one_queen_per_row_txt_file, 8)


Generating and Saving all 40320 possible 8-Queens placements (one per row/column)
Successfully wrote 40320 solutions to 'AllSolutions_8Queens.txt'
Finished generating and saving all 8-Queens placements.

 Validating Solutions from 'AllSolutions_8Queens.txt' for N = 8
Validation Summary for 'AllSolutions_8Queens.txt':
Total Valid Solutions: 92
Total Invalid Solutions: 40228


In [104]:
### DO: Make a function to write a (new) text file containing arrangements of N-queens on a board s.t. no queens attack each other. 
### Use your first function to test this one. Do not use this to prove the first one works. 
### Name your files s.t. it declares the value of N in the name. 

def safe(row,col,l,n):

    trow=row
    tcol=col
    while row>=0 and col>=0:
        if l[row][col]=="Q":
            return False
        row-=1
        col-=1
    row=trow
    col=tcol
    while col>=0:
        if l[row][col]=="Q":
            return False
        col-=1
    col=tcol
    while row<n and col>=0:
        if l[row][col]=="Q":
            return False
        row+=1
        col-=1
    return True

def solve(col,m,l,n):
    if col==n:
        m.append(l[:])
        return
    for row in range(n):
        if safe(row,col,l,n):
            l[row]=l[row][:col]+"Q"+l[row][col+1:]
            solve(col+1,m,l,n)
            l[row]=l[row][:col]+"."+l[row][col+1:]

def solveNQueens(n: int):
    m=[]
    l=['.'*n for _ in range(n)]
    solve(0,m,l,n)
    print(len(m))
    return m

def write_n_queens_to_file(n: int, solutions: list):
    filename = f"NQueens_solutions_{n}.txt"
    try:
        with open(filename, 'w') as f:
            if not solutions:
                f.write("No solutions found.\n")
            else:
                for i, board in enumerate(solutions):
                    for row_str in board:
                        f.write(row_str + "\n")
                    f.write("\n")
        print(f"Successfully wrote {len(solutions)} solutions to '{filename}'")
    except IOError as e:
        print(f"Error writing to file '{filename}': {e}")
    
    return filename

queens_solutions = solveNQueens(N)
N_QUEENS_TXT_FILE = write_n_queens_to_file(N, queens_solutions)

modified_read_and_validate_n_queens_solutions(N_QUEENS_TXT_FILE, 8)



92
Successfully wrote 92 solutions to 'NQueens_solutions_8.txt'

 Validating Solutions from 'NQueens_solutions_8.txt' for N = 8
Validation Summary for 'NQueens_solutions_8.txt':
Total Valid Solutions: 92
Total Invalid Solutions: 0


In [103]:
### DO: Check your counts agree vs. known counts.
def count_n_queens_solutions(n):
    def solve(row, cols, diag1, diag2):
        nonlocal count
        if row == n:
            count += 1
            return
        bits = ~(cols | diag1 | diag2) & ((1 << n) - 1)
        while bits:
            p = bits & -bits
            bits ^= p
            solve(row + 1, cols | p, (diag1 | p) << 1, (diag2 | p) >> 1)

    count = 0
    solve(0, 0, 0, 0)
    return count

# Known correct number of solutions for N = 1 to 10
known_counts = {
    1: 1,
    2: 0,
    3: 0,
    4: 2,
    5: 10,
    6: 4,
    7: 40,
    8: 92,
    9: 352,
    10: 724
}

print(f"{'N':>2} | {'Found':>6} | {'Expected':>8} | {'Match'}")
print("-" * 33)

for n in range(1, 11):
    found = count_n_queens_solutions(n)
    expected = known_counts[n]
    match = "Yes" if found == expected else "No"
    print(f"{n:>2} | {found:>6} | {expected:>8} | {match}")

 N |  Found | Expected | Match
---------------------------------
 1 |      1 |        1 | Yes
 2 |      0 |        0 | Yes
 3 |      0 |        0 | Yes
 4 |      2 |        2 | Yes
 5 |     10 |       10 | Yes
 6 |      4 |        4 | Yes
 7 |     40 |       40 | Yes
 8 |     92 |       92 | Yes
 9 |    352 |      352 | Yes
10 |    724 |      724 | Yes


## Ex. 2 - Qubic

*Tic-tac-toe* is a game played on a $3\times 3$ grid, where player 1 plays X's and player 2 plays O's.   
Players take turn putting X's and O's in the grid.  
The winner is the first player to make 3-in-a-row either horizontally, vertically, or diagonally.  

*Qubic* is tic-tac-toe played in 3-dimensions, where the winning line may also go across the depth of the grid.  
When it was produced, it was played on a $4 \times 4 \times 4$ grid.  
For simplicity, consider only the $3 \times 3 \times 3$ variant.  



In [31]:
### DO: Write a function to list all the valid games of tic-tac-toe in a separate text file




In [32]:
### DO: Write a function to list all the valid games of qubic in a separate text file




In [33]:
### DO: Determine if there is a known winner for both tic-tac-toe and qubic, assuming perfect play.




## Ex. 3 - Queen Coloring 

The set of legal queen moves per square on a chess board defines a graph known as the queen's graph.  
The smallest number of colours needed to make a proper coloring of this graph for an $8\times 8$ board is known to be $9$.  

In [ ]:
### DO: Make a function to find a valid 9-coloring of the queen's graph on an 8x8 board. 
_call_count = 0

def check_conflicts(board, r, c, color, N):

    # Check current row to the left
    for col_idx in range(c):
        if board[r][col_idx] == color:
            return False

    # Check current column upwards
    for row_idx in range(r):
        if board[row_idx][c] == color:
            return False

    # Check main diagonal (top-left to bottom-right) upwards-left
    i, j = r - 1, c - 1
    while i >= 0 and j >= 0:
        if board[i][j] == color:
            return False
        i -= 1
        j -= 1

    # Check anti-diagonal (top-right to bottom-left) upwards-right
    i, j = r - 1, c + 1
    while i >= 0 and j < N:
        if board[i][j] == color:
            return False
        i -= 1
        j += 1
    
    return True # No conflicts found with already placed colors

def solve_queens_coloring_with_progress(N, colors_range, board, row, col):
    global _call_count
    _call_count += 1

    # Base case: If all rows have been successfully colored, a solution is found.
    if row == N:
        return True

    # Calculate the next cell to process
    next_row = row
    next_col = col + 1
    if next_col == N: # Move to the next row if current column is exhausted
        next_row += 1
        next_col = 0

    # Display progress: Show the current cell being attempted
    # Update every 10000 calls to reduce I/O overhead and make it more visible
    if _call_count % 10000 == 0:
        sys.stdout.write(f'\rAttempting cell ({row},{col})... Calls: {_call_count}')
        sys.stdout.flush()

    # Try placing each color from 1 to colors_range
    for color in range(1, colors_range + 1):
        # Check if the current color can be placed without conflicts
        if check_conflicts(board, row, col, color, N):
            board[row][col] = color # Place the color

            # Recursively call for the next cell
            if solve_queens_coloring_with_progress(N, colors_range, board, next_row, next_col):
                return True # If a solution is found in subsequent calls, propagate True
            
            # Backtrack: If the current color choice didn't lead to a solution,
            # reset the cell and try the next color.
            board[row][col] = 0 
    
    return False # No color worked for the current cell, so backtrack

def find_queens_coloring_solution():

    global _call_count
    _call_count = 0 # Reset call count for each run

    N = 8 # Board size (8x8)
    COLORS_RANGE = 9 # Colors from 1 to 9

    # Initialize the board with zeros (0 indicates an uncolored cell)
    board = [[0 for _ in range(N)] for _ in range(N)]

    print("Starting search for 9-coloring of 8x8 Queen's Graph...")
    
    start_time = time.time()
    # Start the backtracking process from the first cell (0,0)
    if solve_queens_coloring_with_progress(N, COLORS_RANGE, board, 0, 0):
        end_time = time.time()
        print(f"\n\nSolution Found in {end_time - start_time:.2f} seconds after {_call_count} calls:")
        # Print the colored board in a matrix format
        for r_idx, row_data in enumerate(board):
            # Format each number to take 2 spaces for alignment
            print(" ".join(f"{cell:2}" for cell in row_data))
        return board
    else:
        end_time = time.time()
        print(f"\n\nNo solution found after {end_time - start_time:.2f} seconds and {_call_count} calls.")
        return None

    
find_queens_coloring_solution()
    

In [45]:
### DO: Prove the colouring is proper, and the graph is a queen's graph. 

    


## Ex. 4 - Knight's Tour

The set of legal knight moves per square on a chess board defines a graph known as the knight's graph.  
A tour is a sequence of edges/vertices that, if traversed, visits each vertex exactly once (i.e. without doubling back).  


In [ ]:
### DO: Make a function to find a sequence of knight moves on an $8\times 8$ chess board s.t. all squares are visited only once.  
### Let it start from any square on the board.  
### The tour may be open or closed (i.e. it does not need to loop back to itself as in TSP). 

def initboard(size):
    board = np.full((size, size), 0, dtype=int)
    return board

def knights_tour(x, y, board, moves):
    size = len(board)
    knight = [(1, 2), (2, 1), (-1, 2), (-2, 1), (1, -2), (2, -1), (-2, -1), (-1, -2)]
    def valid_check(x, y, board):
        if 0 <= x < size and 0 <= y < size and board[x][y] == 0:
            return True
        else:
            return False
    def count_moves(x, y, board):
        count = 0
        for i, j in knight:
            nx, ny = x+i, y+j
            if valid_check(nx, ny, board):
                count += 1
        return count
    board[x][y] = moves
    if moves == size ** 2:
        return True
    next_move = []
    for i, j in knight:
        nx, ny = x + i, y + j
        if valid_check(nx, ny, board):
            move_forward = count_moves(nx, ny, board)
            next_move.append(((nx, ny), move_forward))
    next_move.sort(key=lambda x:x[1])
    for (nx,ny), _ in next_move:
        if knights_tour(nx, ny, board, moves+1):
            return True
        board[nx][ny]=0
    return False

In [49]:
# DO: Prove that your method visits all squares exactly once

Test_Cases_Knights = 20

for test in range(Test_Cases_Knights):
    board = initboard(8)
    start_x, start_y = np.random.randint(0, 8), np.random.randint(0, 8)
    print("Starting x and y:", start_y, "&", start_x)
    if knights_tour(start_x, start_y, board, 1):
        print("Knight's Tour found:")
        print(board, "\n")
    else:
        print("No tour found.")

Starting x and y: 3 & 4
Knight's Tour found:
[[12 59 14 39 10 37 24 33]
 [15 40 11 60 25 34  9 36]
 [58 13 64 41 38 43 32 23]
 [63 16 55 44 61 26 35  8]
 [54 57 62  1 42 31 22 27]
 [17 48 51 56 45 28  7  4]
 [50 53 46 19  2  5 30 21]
 [47 18 49 52 29 20  3  6]] 

Starting x and y: 4 & 2
Knight's Tour found:
[[51 14 57 44 53 16 21 42]
 [56 45 52 15 58 43  2 17]
 [13 50 63 54  1 20 41 22]
 [46 55 32 49 62 59 18  3]
 [33 12 47 64 19 40 23 60]
 [28 31 34 37 48 61  4  7]
 [11 36 29 26  9  6 39 24]
 [30 27 10 35 38 25  8  5]] 

Starting x and y: 6 & 2
Knight's Tour found:
[[29 26  7 62 21 24  5  2]
 [ 8 61 28 25  6  3 20 23]
 [27 30 63 42 59 22  1  4]
 [64  9 60 49 52 43 40 19]
 [31 54 51 58 41 48 15 44]
 [10 57 34 53 50 45 18 39]
 [35 32 55 12 37 16 47 14]
 [56 11 36 33 46 13 38 17]] 

Starting x and y: 1 & 3
Knight's Tour found:
[[23 32  3 64 25 30  5 40]
 [ 2 55 24 31  4 41 26 29]
 [33 22 61 44 63 28 39  6]
 [54  1 56 35 60 45 42 27]
 [21 34 53 62 43 36  7 38]
 [52 49 18 57 46 59 10 13]
 